# ***Eğitim Performansı***

## ***Ev fiyatları modeli***

Bu alıştırmada, ev fiyatları modelinizde çalışacaksınız. İlk olarak, Ev Fiyatları verilerini [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)'dan indirin.

- Ev fiyatları modelinizi tekrar çalıştırın ve F-testi, R-kare, ayarlanmış R-kare, AIC ve BIC kullanarak modelinizin uygunluğunu değerlendirin. Modelinizin tatmin edici olduğunu düşünüyor musunuz? Açıklayın

- Modelinizin uygunluğunu iyileştirmek için, bazı değişkenleri ekleyerek veya kaldırarak farklı model özelliklerini deneyin.

- Denediğiniz her model için, uygun metrikleri alın ve modellerinizi birbiriyle karşılaştırın. Hangi model en iyisidir ve neden?

---

In [149]:
import pandas as pd
import numpy as np
# grafiksel araclar
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
# lineer regresyon icin
from sklearn import linear_model
import statsmodels.api as sm
# polynomial features icin
from sklearn.preprocessing import PolynomialFeatures
# path ile calismak icin
import sys
# dosyanin bulundugu dizini path e ekledik
mylib = '/home/admin1/datascience/datascience/library'
if mylib not in sys.path:
    sys.path.insert(1, mylib)
# parser i kullanabilmek icin
from data_description_parser import data_description_house_prices
data_dict = data_description_house_prices()

# uyarilari dikkate alma
import warnings
warnings.filterwarnings('ignore')

# pandas varsayilan olarak cok sayida sutun veya satir varsa tumunu gostermez
# bu nedenle 100 sutun ve satir gostermesi icin
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# virgulden sonra belirli sayida basamak gosterimi icin 
pd.options.display.float_format = '{:,.3f}'.format
# aciklama sutunun tam cikmasi icin
pd.options.display.max_colwidth = 100

# font tanimlamalari
title_font = {'family': 'times new roman', 'color': 'darkred','weight': 'bold','size': 14}
axis_font  = {'family': 'times new roman', 'color': 'darkred','weight': 'bold','size': 14}

# temiz veri setini dataframe icerisine yukle (outlierlarin bir kismi temizlenmis)
house_prices = pd.read_csv('data/house_prices_out.csv',index_col='Id')

- Sonuclar dataframe olusturularak kaydedilecek.

In [150]:
# sonuclari tutacak df
results_df = pd.DataFrame(columns=['description', 'rsquared','rsquared_adj', 'fvalue', 'aic', 'bic'])
results_df.index.name = 'model' 


In [151]:
# sonuclari kaydeden fonksiyon
def save_results(results, model_nu, description):
    global results_df
    df = pd.DataFrame([])
    df.index.name = 'model'
    if results_df.shape[0] == model_nu-1:
        df.loc[model_nu,'description'] = description
        df.loc[model_nu,'rsquared'] = results.rsquared
        df.loc[model_nu,'rsquared_adj'] = results.rsquared_adj
        df.loc[model_nu,'fvalue'] = results.fvalue
        df.loc[model_nu,'aic'] = results.aic
        df.loc[model_nu,'bic'] = results.bic    
        
        results_df = pd.concat([results_df, df])
    

### ***1. Model (Numerik degiskenlerle)***

- Numerik degiskenleri ayiralim. SalePrice ayrildiktan sonra toplam 36 adet numerik degisken mevcut.

In [152]:
numeric_vars = [var for var in house_prices.columns if house_prices[var].dtype != 'object']
numeric_vars.remove('SalePrice')
len(numeric_vars)

# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[numeric_vars]

# statsmodel kullanarak lrm
# bias (intercept) değişkenini manuel olarak ekliyoruz
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# 1 inci model sonuclarini kaydedelim
save_results(results, 1, 'numerik degiskenler')

# 1 inci model sonuclari
results_df

,description,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,,
1,numerik degiskenler,0.867,0.864,270.643,"33,564.671","33,749.253"


### ***2. Model (Yuksek korelasyonlu numerik degiskenlerle)***

In [153]:
# yuksek korelasyonlu surekli degiskenler
high_corr = ['1stFlrSF', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual']

# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[high_corr]

# statsmodel kullanarak lrm
# bias (intercept) değişkenini manuel olarak ekliyoruz
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# 2 inci model sonuclarini kaydedelim
save_results(results, 2, 'yuksek korelasyonlu numerik degiskenler')

# 2 inci model sonuclari
results_df

,description,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,,
1,numerik degiskenler,0.867,0.864,270.643,"33,564.671","33,749.253"
2,yuksek korelasyonlu numerik degiskenler,0.804,0.803,980.924,"34,072.076","34,108.992"


### ***3. Model (Farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular)***

In [154]:
# neighborhood degiskenini encode edecek fonksiyon
def encode_neighborhood(val):
    # belirgin sekilde en yuksek fiyat ozelligi gosteren neighborhood degerleri
    if val in ['NoRidge', 'NridgHt',  'StoneBr']:
        return 1
    else:
        return 0

# encode edilen degisken ayri sutunda
house_prices['Neighborhood_e'] = house_prices['Neighborhood'].apply(encode_neighborhood)
# lambda ile 'Partial' olan evlere 1 digerlerine 0 atadik
house_prices['SaleCondition_e'] = house_prices['SaleCondition'].apply(lambda x: 
                                                                      1 if x == 'Partial' else 0)

# farklilik olusturan kategorikler eklendi
expl_vars = high_corr + ['Neighborhood_e', 'SaleCondition_e'] 

# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]

# statsmodel kullanarak lrm
# bias (intercept) değişkenini manuel olarak ekliyoruz
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# 3 inci model sonuclarini kaydedelim
save_results(results, 3, 'farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular')

# 3 inci model sonuclari
results_df

,description,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,,
1,numerik degiskenler,0.867,0.864,270.643,"33,564.671","33,749.253"
2,yuksek korelasyonlu numerik degiskenler,0.804,0.803,980.924,"34,072.076","34,108.992"
3,farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.832,0.831,887.437,"33,853.154","33,900.618"


### ***4. Model Etkilesimli(Farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular)***

In [155]:
# farklilik olusturan kategorikler eklendi
expl_vars = high_corr + ['Neighborhood_e', 'SaleCondition_e'] 

# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]

# kacinci dereceden polinomlar olusturacagiz belirtiyoruz
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

# statsmodel kullanarak lrm
# bias (intercept) değişkenini manuel olarak ekliyoruz
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# 4 inci model sonuclarini kaydedelim
save_results(results, 4, 'etkilesimli(farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular')

# 4 inci model sonuclari
results_df

,description,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,,
1,numerik degiskenler,0.867,0.864,270.643,"33,564.671","33,749.253"
2,yuksek korelasyonlu numerik degiskenler,0.804,0.803,980.924,"34,072.076","34,108.992"
3,farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.832,0.831,887.437,"33,853.154","33,900.618"
4,etkilesimli(farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.874,0.870,231.100,"33,506.517","33,733.289"


### ***5. Model (Butun degiskenlerle)***

In [156]:
# temiz veri setini dataframe icerisine yukle (outlierlarin bir kismi temizlenmis)
house_prices = pd.read_csv('data/house_prices_out.csv',index_col='Id')

# object tipinde olanlar kategorik degiskenler listesine ayir
qualitative = [column for column in house_prices.columns if house_prices.dtypes[column] == 'object']

# object tipinde olanlar kategorik degiskenler listesine ayir
quantitative = [column for column in house_prices.columns if house_prices.dtypes[column] != 'object']

def encode(df, feature):
    ordering = pd.DataFrame()
    ordering['val'] = df[feature].unique()
    # degerler index yapildi
    ordering.index = ordering.val
    # degisken degerlerinin SalePrice ortalamalarini aliyor
    ordering['spmean'] = df[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    # ortalamalari siraliyor
    ordering = ordering.sort_values('spmean')
    # siralama degerleri ayri bir sutunda
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    #print(ordering)
    ordering = ordering['ordering'].to_dict()    
    # en son soyle bir sozluk olusur ExterQual icin
    # {'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
    
    # burasi cok ilginc yeni encoded degiskeni burada olusturuyor
    for cat, o in ordering.items():        
        df.loc[df[feature] == cat, feature+'_E'] = o
        
qual_encoded = []
for q in qualitative:  
    encode(house_prices, q)
    qual_encoded.append(q+'_E')

quantitative.remove('SalePrice')

expl_vars = quantitative + qual_encoded
# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]

# statsmodel kullanarak lrm
# bias (intercept) değişkenini manuel olarak ekliyoruz
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# 5 inci model sonuclarini kaydedelim
save_results(results, 5, 'butun degiskenler')

# 5 inci model sonuclari
results_df

,description,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,,
1,numerik degiskenler,0.867,0.864,270.643,"33,564.671","33,749.253"
2,yuksek korelasyonlu numerik degiskenler,0.804,0.803,980.924,"34,072.076","34,108.992"
3,farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.832,0.831,887.437,"33,853.154","33,900.618"
4,etkilesimli(farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.874,0.870,231.100,"33,506.517","33,733.289"
5,butun degiskenler,0.902,0.896,162.398,"33,219.548","33,630.903"


### ***6. Model (Butun degiskenlerden anlamli olmayanlar cikarilarak)***

- pval degerine gore iststistiki anlami olamayanlar cikarilir.

In [157]:
vars_to_remove = []
# p degerleri yuksek olanlari cikar
for var in expl_vars:
    if results.pvalues[var]>0.05:
        vars_to_remove.append(var)
        
# istenmeyenleri ayir
for var in vars_to_remove:
    if var in expl_vars:
        expl_vars.remove(var)
       

# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# 6 inci model sonuclarini kaydedelim
save_results(results, 6, 'butun degiskenlerden anlamli olmayanlar cikarilarak')

# 6 inci model sonuclari
results_df

,description,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,,
1,numerik degiskenler,0.867,0.864,270.643,"33,564.671","33,749.253"
2,yuksek korelasyonlu numerik degiskenler,0.804,0.803,980.924,"34,072.076","34,108.992"
3,farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.832,0.831,887.437,"33,853.154","33,900.618"
4,etkilesimli(farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.874,0.870,231.100,"33,506.517","33,733.289"
5,butun degiskenler,0.902,0.896,162.398,"33,219.548","33,630.903"
6,butun degiskenlerden anlamli olmayanlar cikarilarak,0.897,0.895,371.777,"33,197.428","33,376.737"


### ***7. Model Etkilesimli(Butun degiskenlerden anlamli olmayanlar cikarilarak)***

In [158]:
# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]

# kacinci dereceden polinomlar olusturacagiz belirtiyoruz
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# 7 inci model sonuclarini kaydedelim
save_results(results, 7, 'etkilesimli(butun degiskenlerden anlamli olmayanlar cikarilarak)')

# 7 inci model sonuclari
results_df


,description,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,,
1,numerik degiskenler,0.867,0.864,270.643,"33,564.671","33,749.253"
2,yuksek korelasyonlu numerik degiskenler,0.804,0.803,980.924,"34,072.076","34,108.992"
3,farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.832,0.831,887.437,"33,853.154","33,900.618"
4,etkilesimli(farklilik olusturan kategorik degiskenler ve yuksek korelasyonlular,0.874,0.870,231.100,"33,506.517","33,733.289"
5,butun degiskenler,0.902,0.896,162.398,"33,219.548","33,630.903"
6,butun degiskenlerden anlamli olmayanlar cikarilarak,0.897,0.895,371.777,"33,197.428","33,376.737"
7,etkilesimli(butun degiskenlerden anlamli olmayanlar cikarilarak),0.965,0.943,43.471,"32,696.765","35,644.812"


### ***Degerlendirme***

- 1inci modelde butun numerik degiskenler mevcut r-kare degeri tatmin edici ancak f degeri diger modellere nazaran cok dusuk,

- 2inci modelde yuksek korelasyonlu numerik degiskenleri kullandik r-kare degerleri dustu ancak f degeri diger modellerin hepsinden iyi, aic ve bic degeri ise diger modellere nazaran yuksek,

- 3uncu modelde yuksek korelasyonlu degiskenlere iki adet farklilik olusturdugunu dusundugumuz kategorik degisken ekledik r-kare degerimiz 2. modele gore artti, f degeri biraz dustu(olumsuz), aic ve bic degerleri de dustu(olumlu),

- 4uncu modelde 3 uncu modeldeki degiskenlerle polynomial features ozelligini kullandik, r-kare degerleri beklendigi gibi yukseldi ancak f degeri cok dustu(olumsuz), aic ve bic degeri biraz dustu(olumlu),

- 5inci modelde ise butun degiskenleri(surekli ve kategorik) modele kattik, r-kare degeri cok yukseldi ancak f degeri en dusuk ikinci model(olumsuz), aic ve bic degerleri en dusuk ikinci(olumlu),

- 6inci modelde 5.modeldeki istatistiki anlami olmayan (pval>0.05) degiskenleri cikarttik, r-kare degeri beklendigi gibi cok dusmedi ve f degerini arttirmis olduk(en iyi ucuncu), aic ve bic degerleri de en dusuk(olumlu) modeli elde ettik,

- 7inci modelde 6.modeldeki degiskenlerle polynomial features ozelligini kullandik, r-kare degerleri beklendigi gibi cok yukseldi ancak f degeri cen dusuk model(olumsuz), aic degeri en dusuk deger(olumlu) ancak bic en yuksek model(olumsuz) cunku bic aciklayici degisken sayisi yuksek olan modeli cezalandiriyor. 